In [2]:
import numpy as np
#import mnist
import keras
from keras.datasets import fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [5]:
print(train_images.shape) # (60000, 28, 28)
print(test_images.shape)  # (10000, 28, 28)

(60000, 28, 28)
(10000, 28, 28)


In [6]:
# Flatten the images.
train_images = train_images.reshape((-1, train_images.shape[1] * train_images.shape[2]))
test_images = test_images.reshape((-1, test_images.shape[1] * test_images.shape[2]))

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 784)
(10000, 784)


In [7]:
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [8]:
from keras.models import Sequential
from keras.layers import Dense 
from keras.utils import to_categorical
from keras.datasets import mnist

In [9]:
model = Sequential()
model.add(Dense(64, input_shape=(784,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

In [10]:
# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics='accuracy',
)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Train the model.
met = model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=2,
  batch_size=200,
)

print(met)

# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Predict on the first 5 test images.
predictions = model.predict(test_images)

# Print our model's predictions.
print(np.argmax(predictions, axis=1)[:10]) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:10]) # [7, 2, 1, 0, 4]

Epoch 1/2
300/300 [==============================] - 1s 3ms/step - loss: 0.6720 - accuracy: 0.7570
Epoch 2/2
313/313 [==============================] - 0s 1ms/step - loss: 0.4462 - accuracy: 0.8318
[9 2 1 1 6 1 4 6 5 7]
[9 2 1 1 6 1 4 6 5 7]


In [13]:
met = model.evaluate(
  test_images,
  to_categorical(test_labels)
)

313/313 [==============================] - 0s 969us/step - loss: 0.4462 - accuracy: 0.8318


In [14]:
met

[0.44624078273773193, 0.8317999839782715]

In [40]:
activation_functions = ['relu', 'elu', 'tanh', 'sigmoid', 'exponential', 'linear']
num_layers = [1, 2, 3]
num_neuron = [20, 30, 50, 70, 100]
learning_rates = [0.1, 0.01, 0.001, 'adam']
epoch = [2, 5, 10, 20]
batch_size = [5, 20, 30, 60, 100, 200, 500]

In [41]:
def model_init(model, num_layers, num_neuron, func, lr):
  #model = Sequential()
  
  if lr != 'adam':
    opt = keras.optimizers.Adam(learning_rate=lr)
  else:
    opt = 'adam'

  model.add(Dense(num_layers, input_shape=(784,), activation=func))

  for i in range(num_layers):
    model.add(Dense(num_neuron, activation=func))

  model.add(Dense(10, activation='sigmoid'))

  # Compile the model.
  model.compile(
    #optimizer='adam',
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics='accuracy',
  )



def model_fit(model, epochs, batch_size): 
  # Train the model.
  model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=epochs,
    batch_size=batch_size,
  )

  # Evaluate the model.
  metric = model.evaluate(
    train_images,
    to_categorical(train_labels)
  )

  return metric

In [ ]:
%%time

repeat = 5
result = list()


for af in activation_functions:
  for nl in num_layers:
    for nn in num_neuron:
      for lr in learning_rates:
        for e in epoch:
          for bs in batch_size:
            result_dict = dict()
            result_dict['learning_rate'] = lr
            result_dict['activation_function'] = af
            result_dict['num_layer'] = nl
            result_dict['num_neuron'] = nn
            result_dict['epochs'] = e
            result_dict['batch_size'] = bs
            list_accuracy = list()
            list_loss = list()
            print(f'activation_function:{af}, num_layer:{nl}, epochs:{e}, batch_size:{bs}, learning_rates:{lr}')
            for r in range(repeat):  
              if r > 0 and list_accuracy[-1] < 0.8:
                break
              model = Sequential()
              model_init(model, nl, nn, af, lr)
              buf = model_fit(model, e, bs)
              list_accuracy.append(buf[1])
              list_loss.append(buf[0])
            
            result_dict['accuracy'] = np.array(list_accuracy).mean()
            result_dict['loss'] = np.array(list_loss).mean()

            result_dict['accuracy_list'] = list_accuracy
            result_dict['loss_list'] = list_loss

            result.append(result_dict)

In [31]:
#Это оказалось долго и не информативно
#Пойдем по кускам

In [55]:
def fit_for_parameters(result, af, lr, nl, nn, e, bs, repeat=10):
  result_dict = dict()
  result_dict['activation_function'] = af
  result_dict['learning_rate'] = lr  
  result_dict['num_layer'] = nl
  result_dict['num_neuron'] = nn
  result_dict['epochs'] = e
  result_dict['batch_size'] = bs
  list_accuracy = list()
  list_loss = list()
  print(f'activation_function:{af}, num_layer:{nl}, epochs:{e}, batch_size:{bs}, learning_rates:{lr}, num_neuron:{nn}')
  for r in range(repeat):  
    #if r > 0 and list_accuracy[-1] < 0.5:
      #break
    model = Sequential()
    model_init(model, nl, nn, af, lr)
    buf = model_fit(model, e, bs)
    list_accuracy.append(buf[1])
    list_loss.append(buf[0])
  
  result_dict['accuracy'] = np.array(list_accuracy).mean()
  result_dict['loss'] = np.array(list_loss).mean()

  result_dict['accuracy_list'] = list_accuracy
  result_dict['loss_list'] = list_loss

  result.append(result_dict)

def best_search(result, parameter):
  accuracy__ = 0;

  for i in range(len(result)):
    print(result[i][parameter], ':', result[i]['accuracy'])
    
    if result[i]['accuracy'] > accuracy__:
      accuracy__ = result[i]['accuracy']
      i_best = i

  print()
  print('best: ', end=' ')
  print(result[i_best][parameter])

In [ ]:
#ВЫберем активационную функцию
%%time

result = list()

lr = 0.001
nl = 1
nn = 64
e = 5
bs = 200

for af in activation_functions:
  fit_for_parameters(result, af, lr, nl, nn, e, bs)  

In [56]:
best_search(result, 'activation_function')


relu : 0.371668329834938
elu : 0.2625933334231377
tanh : 0.26429333388805387
sigmoid : 0.25866000205278394
exponential : 0.18986667022109033
linear : 0.3025616675615311

best:  relu


In [ ]:
#Лучшая релу, берем еЁ.
#Поиск скорости обучения 
%%time

result = list()

af='relu'
lr = 0.001
nl = 1
nn = 64
e = 5
bs = 200

for lr in learning_rates:
  fit_for_parameters(result, af, lr, nl, nn, e, bs) 

In [58]:
best_search(result, 'learning_rate')

0.1 : 0.170021665841341
0.01 : 0.43616833388805387
0.001 : 0.3099966675043106
adam : 0.3343383356928825

best:  0.01


In [59]:
#Странно, что даже с оптимизатором результат хуже, чем с фиксированной скоростью обучения, но ладно берем ее. Хотя есть вариант, что при 0.001 она не успевала до обучиться.

In [ ]:
#Подберем количество слоев, помимо входного и выходного
#Лучшая релу, берем еЁ.
#Поиск скорости обучения 
%%time

result = list()

af='relu'
lr = 0.01
nl = 1
nn = 64
e = 5
bs = 200

for nl in num_layers:
  fit_for_parameters(result, af, lr, nl, nn, e, bs) 

In [61]:
best_search(result, 'num_layer')

1 : 0.40863500311970713
2 : 0.7114233314990998
3 : 0.7765216708183289

best:  3


In [62]:
#Лучший результат с 3 внутр. слоями

In [ ]:
#Подберем кол-во нейронов в слое
%%time

result = list()

af='relu'
lr = 0.01
nl = 3
nn = 64
e = 5
bs = 200

for nn in num_neuron:
  fit_for_parameters(result, af, lr, nl, nn, e, bs) 

In [65]:
best_search(result, 'num_neuron')

20 : 0.7880733430385589
30 : 0.7653849959373474
50 : 0.7731266677379608
70 : 0.7026816695928574
100 : 0.7426766753196716

best:  20


In [66]:
#Лидирует 20 нейронов. Сложно сделать вывод просто возьму 20 нейронов.

In [ ]:
#Подберем кол-во эпох
%%time

result = list()

af='relu'
lr = 0.01
nl = 3
nn = 20
e = 5
bs = 200

for e in epoch:
  fit_for_parameters(result, af, lr, nl, nn, e, bs) 

In [68]:
best_search(result, 'epochs')

2 : 0.6851583316922187
5 : 0.7400266647338867
10 : 0.7877966642379761
20 : 0.7945316672325134

best:  20


In [69]:
#Результат логичен, поскольку кол-во эпох это количество проходов по нашей тренировочной выборке, то с увеличением кол-ва проходов модель сильнее обучается тренировочной выборке, при этом мы не контролируем результат тестовой выборки. Не будем брать 20, но возьмем 10.


In [ ]:
#Подберем данных, после обработки которых меняем веса
%%time

result = list()

af='relu'
lr = 0.01
nl = 3
nn = 20
e = 10
bs = 200

for bs in batch_size:
  fit_for_parameters(result, af, lr, nl, nn, e, bs, repeat=3) 

In [74]:
best_search(result, 'batch_size')

5 : 0.5757111112276713
20 : 0.7276888887087504
30 : 0.7800944447517395
60 : 0.7791222135225931
100 : 0.7919833262761434
200 : 0.7699388861656189

best:  100


In [85]:
%%time

result = list()

af='relu'
lr = 0.01
nl = 3
nn = 20
e = 10
bs = 100

fit_for_parameters(result, af, lr, nl, nn, e, bs, repeat=5) 

activation_function:relu, num_layer:3, epochs:10, batch_size:100, learning_rates:0.01, num_neuron:20
Epoch 1/10
600/600 [==============================] - 1s 2ms/step - loss: 0.8362 - accuracy: 0.6747
Epoch 2/10
600/600 [==============================] - 1s 2ms/step - loss: 0.6435 - accuracy: 0.7700
Epoch 3/10
600/600 [==============================] - 1s 2ms/step - loss: 0.6053 - accuracy: 0.7871
Epoch 4/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5988 - accuracy: 0.7876
Epoch 5/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5874 - accuracy: 0.7944
Epoch 6/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5802 - accuracy: 0.7951
Epoch 7/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5736 - accuracy: 0.7984
Epoch 8/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5655 - accuracy: 0.8001
Epoch 9/10
600/600 [==============================] - 1s 2ms/step - loss: 0.5664 - accuracy: 0.7997

In [86]:
result

[{'accuracy': 0.7868366599082947,
  'accuracy_list': [0.8082000017166138,
   0.7387333512306213,
   0.7870333194732666,
   0.7855833172798157,
   0.814633309841156],
  'activation_function': 'relu',
  'batch_size': 100,
  'epochs': 10,
  'learning_rate': 0.01,
  'loss': 0.5822299480438232,
  'loss_list': [0.5550279021263123,
   0.6941044926643372,
   0.5808365345001221,
   0.5669528245925903,
   0.5142279863357544],
  'num_layer': 3,
  'num_neuron': 20}]

In [ ]:
#Самы лучший результат accuracy:0.814 loss:0.514. Вообще по идее увеличивая batch_size должна расти точность. Скорее всего на batch_size больше 100 не хватает эпох для лучшего обучения.